<a href="https://colab.research.google.com/github/anjali-ojha/deep-learning/blob/a10/assignment/assignment-10/Anjali_Ojha_HW_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT for QnA:

# 1. Load the SQUAD 2.0 (https://pytorch.org/text/stable/datasets.html#torchtext.datasets.SQuAD2) dataset from pytorch dataset library. take minimum 20 QnA pairs.




In [2]:
! pip install datasets
! pip install portalocker
! pip install transformers datasets torch
! pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-n

In [3]:
from datasets import load_dataset

def load_data():
  # Load the SQuAD 2.0 dataset
  squad_dataset = load_dataset("squad_v2")

  # Get the training data
  train_data = squad_dataset["train"]

  # Get the development data
  dev_data = squad_dataset["validation"]
  return train_data, dev_data


train_data, dev_data = load_data()
train_data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 130319
})

In [4]:
train_data.features

{'id': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}

In [5]:
train_data['answers'][:2]

[{'text': ['in the late 1990s'], 'answer_start': [269]},
 {'text': ['singing and dancing'], 'answer_start': [207]}]

# 2. Display a few raw QnA data samples.


In [6]:
for i in range(5):
  print(f"\nsample = {i+1}, \n\t Ques = {train_data['question'][i]}, \n\t Ans  = {train_data['answers'][i]['text'][0]}")
# train_data['question'][:2]


sample = 1, 
	 Ques = When did Beyonce start becoming popular?, 
	 Ans  = in the late 1990s

sample = 2, 
	 Ques = What areas did Beyonce compete in when she was growing up?, 
	 Ans  = singing and dancing

sample = 3, 
	 Ques = When did Beyonce leave Destiny's Child and become a solo singer?, 
	 Ans  = 2003

sample = 4, 
	 Ques = In what city and state did Beyonce  grow up? , 
	 Ans  = Houston, Texas

sample = 5, 
	 Ques = In which decade did Beyonce become famous?, 
	 Ans  = late 1990s


# 3. Preprocess/Clean dataset as per BERT format.

In [7]:
from transformers import BertTokenizer, BertForQuestionAnswering

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

N = 1000

def preprocess_qa(example):
    question, context, answer = example['question'], example['context'], example['answers']['text'][0]
    inputs = tokenizer(question, context, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    inputs.update({'labels': tokenizer(answer, padding='max_length', truncation=True, max_length=32, return_tensors='pt')['input_ids']})
    return inputs


# Define your filter function
def filter_function(example):
    return len(example['answers']['text']) == 1


train_data, dev_data = load_data()

# Filter the train data
filtered_train_data = train_data.select(range(N)).filter(filter_function)
filtered_train_data
train_data = filtered_train_data.map(preprocess_qa)


# Filter the train data
filtered_dev_data = dev_data.select(range(N)).filter(filter_function)
dev_data = filtered_dev_data.map(preprocess_qa)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
train_data.select(range(100))

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 100
})


# 4. Train the BERT QnA model. Evaluate the model.


In [9]:
train_data.features

{'id': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None),
 'input_ids': Sequence(feature=Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), length=-1, id=None),
 'labels': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None)}

In [17]:
import torch
from transformers import BertForQuestionAnswering, BertTokenizer
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from datasets import load_metric

# Initialize BERT and tokenizer
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_data) * 3)

import torch

# Check if CUDA is available (i.e., GPU is available)
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

print("Device:", device)

def train_epoch(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        print(batch['input_ids'])
        input_ids = batch['input_ids'].to(device)  # Assuming 'input_ids' is a list containing tensors
        attention_mask = batch['attention_mask'][0][0].to(device)  # Assuming 'attention_mask' is a list containing tensors
        start_positions = batch['answer_start'][0][0].to(device)  # Assuming 'start_positions' is a list containing tensors
        end_positions = batch['end_positions'][0].to(device)  # Assuming 'end_positions' is a list containing tensors
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
    return total_loss / len(dataloader)


def evaluate_model(model, dataloader, device):
    model.eval()
    metric = load_metric("squad_v2")
    for batch in dataloader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        predictions = tokenizer.batch_decode(torch.argmax(outputs.start_logits, axis=1))
        metric.add_batch(predictions=predictions, references=batch['answers'])
    return metric.compute()


# Set up data loaders
train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)
dev_dataloader = DataLoader(dev_data, batch_size=8, shuffle=False)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(3):  # Number of epochs
    train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, device)
    print(f"Epoch {epoch + 1} - Train Loss: {train_loss:.4f}")

# Evaluate the model on the dev set
evaluation_result = evaluate_model(model, dev_dataloader, device)
print("Evaluation Result:", evaluation_result)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CUDA is available. Using GPU.
Device: cuda
[[tensor([101, 101, 101, 101, 101, 101, 101, 101]), tensor([ 2040,  2044, 25479,  1999,  2054, 20773, 21798,  2054]), tensor([8678, 2054, 2001, 2054, 2112, 2165, 4969, 3538]), tensor([ 2007,  3185, 17351,  3596,  2106,  2188,  1010,  2106]), tensor([20773, 17274,  3141,  1997,  2016,  2129,  2129, 25479]), tensor([2006, 3802, 2000, 2865, 2376, 2116, 2116, 2834]), tensor([1996, 2696, 2054, 2038, 2005, 2982, 2636, 2044]), tensor([ 2309,  2508,  1029, 15296,  1996,  2012,  2004,  4192]), tensor([ 1010,  1010,   102,  2042,  3185,  1996, 20773,  2000]), tensor([ 2139,  2106,  1999,  1996,  8680, 28623,  2853,  8199]), tensor([ 3900, 20773,  2010,  3395,  1029,  8922,  1029,  1029]), tensor([24728,  3443,  3128,  1997,   102,  2982,   102,   102]), tensor([ 1029, 14673,  3735,  1029, 20773,  1029, 20773,  2067]), tensor([  102,  9205,  1010,   102, 11299,   102,  2038,  1999]), tensor([20773,  1029,  1999,  1999,  2006,  2012,  2363,  8199]), tenso

AttributeError: 'list' object has no attribute 'to'

# 5. Perform an Inference and show the predicted vs ground truth answers.